In [1]:
from DELCgen import *
import scipy.stats as st
import matplotlib.pyplot as plt 
from constants import *
from scipy.interpolate import interp1d
#from scipy.integrate import qua
import naima
import astropy.units as u

This piece of code runs a simple simulation where a CR population is injected and evolved with a variable jet. We need to define the maximum rigidity which is related to the jet power
$$
R_{\mathrm{max}} = 10^{19}~V\left(\frac{Q_j}{10^{44}~\mathrm{erg~s}^{-1}}\right)^{1/2}
    \left(\frac{\eta}{0.1}\right)
    \left(\frac{u}{0.1c}\right).
$$

the "power threshold" is the inverse expression with $Q_j$ as the subject. We use an input power spectral density (PSD) and jet power probability distribution function (PDF) to generate a time series for the jet power for a given set of PSD and PDF parameters. To do this, we use the method described by Emmanolopoulous et al. and implemented in python by Connolly. The PSD is implemented as a bending power-law model of the form 

$$\mathrm{PSD}(\nu,\bar{Q},\nu_c) = A(\bar{Q}) \frac{\nu^{-\alpha_{\mathrm{low}}}}
               {1+(\nu/\nu_c)^{\alpha_{\mathrm{high}}-\alpha_{\mathrm{low}}}}$$
               
               
We set $\alpha_{\mathrm{low}} = 1$ for a pink noise spectrum, $\alpha_{\mathrm{high}} = 10$ so as to suppress high frequency variability, and $\nu_c = 1$Myr, roughly equal to the light travel time, $\tau_c=L/c$, along a $L=300$kpc jet. The acceleration time relates to $\tau_c$ by $\tau_\mathrm{acc}/\tau_c \approx (Rc)/(uL)$, since the highest energy particles reach Larmor radii comparable to $(u/c)R$ (Hillas criterion). The longest CR acceleration time is therefore shorter than the light travel time along the jet for any fast, long and thin jet ($u/c>R/L$). 

In [2]:
class Losses:
    def __init__(self, elem_name):
        self.energies, self.total, self.pp, self.pd, self.ep = np.loadtxt("tau_{}.dat".format(elem_name), unpack=True)

    def interpol(self, energies):
        interp_func = interp1d(self.energies, self.total)
        self.total_interpol = interp_func(energies)
        
def power_threshold(rigidity, v_over_c=0.1, eta=0.1):
    power = (0.1 / eta) * (rigidity / 1e19)**2 * (0.1/v_over_c) * 1e44
    return power 

def max_energy(power, v_over_c=0.1, eta=0.1):
    return 1e19*np.sqrt( (power/1e44) * (v_over_c/0.1))

def get_lc(lognorm_params, PSD_params, tbin, Age):
    # Simulation params
    # let's do everything in units of kyr
    # run for 100 Myr (1e5 kyr) in bins of 0.1 Myr
    #lognorm_params = (1.5,0,np.exp(1.5))
    RedNoiseL,RandomSeed,aliasTbin = 100,12,100
    N = Age / tbin

    lc = Simulate_DE_Lightcurve(BendingPL, PSD_params,st.lognorm,lognorm_params,
                                    RedNoiseL=RedNoiseL,aliasTbin=aliasTbin,randomSeed=RandomSeed,LClength=Age, tbin=tbin)

    return (lc)

Now we've set up the basic functions. Let's initialise our parameters for the variable jet history and initialise the tau_loss dictionary which stores the loss times for the different species.  

In [3]:
# set up light curve. pl_index of 1 means red noise.
pl_index = 1

# bend the power law at 1 Myr (1e-3 kyr^-1) and steeply decay it beyond that with index 10. 
# A,v_bend,a_low,a_high,c = 1, 1e-3, pl_index, 10, 1
PSD_params = (1, 1e-3, pl_index, 10, 1)
lognorm_params = (1.5,0,np.exp(1.5))
tbin = 100    # 100 kyr 

# Age of 1000 means 100 Myr
# paramaters for lc are lognorm parameters, PSD parameters, tbin and Age (Age is really number of points)
lc = get_lc(lognorm_params, PSD_params, tbin,1000)

# time is in kyr, so convert to Myr
times = np.arange(0,1000.0*tbin,tbin) 

# get losses 
elems = ["H", "He", "N", "Fe"]
tau_loss = dict()
energies = np.logspace(6,20,num=3000)
for i in range(len(elems)):
    tau_loss[elems[i]] = Losses(elems[i])
    tau_loss[elems[i]].interpol(energies)

Set up arrays to loop over. flux_scales contains the normalisations of our jet power $(\bar{Q})$.
This is actually the median of the distribution, or the mean in log space.
betas is the spectral index of the injected spectrum ($\beta$).

In [4]:
# flux_scales contains the normalisations of our jet power 
# actually the median of the distribution, or the mean in log space.
# betas is the spectral index of the injected spectrum 
flux_scales = np.logspace(43,45,num=20)
betas = np.arange(2,3,0.1)
betas = [2]
flux_scales = [1e43]

# arrays to store gamma ray and UHECR luminosities
lgammas = np.zeros((len(flux_scales), len(betas)))
lcrs = np.zeros((len(flux_scales), len(betas)))

Let's loop over the flux scales and the spectral indices. The method is as follows. For each value of $\bar{Q}$ and $\beta$, we do the following. For each jet power prescription, we evolve a CR distribution for each ionic species numerically using the following equation,

$$
    \frac{dn_i(E)}{dt} = S(E/Z_i, f_i, t) - \frac{n(E)}{\tau_\mathrm{esc} (E/Z_i)} - \frac{n(E)}{\tau_\mathrm{loss} (E, Z_i)}.
$$

The total CR spectrum is then just $n(E,t)=\sum_i n_i(E,t)f_i$. We follow, e.g., \cite{eichmann_ultra-high-energy_2018} in using a power-law for the injection term $S({\cal R}_i, f_i, t)$ with a sharp cutoff, given by

$$
    S(E/Z_i, f_i, t) = 
    f_i \eta Q_j(t)~\nu_I
    \left(\frac{{\cal R}}{{\cal R}_0}\right)^{-\beta}~f_{\mathrm{cut}}(R_{\mathrm{max}}).
    % f_\mathrm{cut}
$$

where $\nu_i$ is the normalisation of the distribution and is given by $\nu_i=1.0/[\ln({\cal R}_\mathrm{max}(t)/{\cal R}_0]$ for $\beta=2$ and 

In [ ]:
for i_flux, flux_scale in enumerate(flux_scales):

    # normalise the light curve
    flux = lc.flux * flux_scale 
    
    # loop over spectral indices
    for i_beta, BETA in enumerate(betas):
    
        # elemental "abundances" - really injection fractions 
        frac_elem = np.array([1.0,0.1,1e-4,3.16e-05])
        
        # normalise! 
        frac_elem /= np.sum(frac_elem)
        
        # charges 
        z_elem = np.array([1,2,7,26])
        
        # array to store the current CR spectrum for each ion
        ncr = np.zeros( (len(z_elem),len(energies)) )
        
        # arrays to store the stored and escaping CRs for every time bin and every ion
        ncr_time = np.zeros( (len(z_elem),len(flux),len(energies)) )
        escaping_time = np.zeros( (len(z_elem),len(flux),len(energies)) )

        # do you want to plot everything? 
        plot_all = True

        # R0 is the injection rigidity
        R0 = 1e9
        
        # Zbar is the average charge 
        Zbar = np.sum(frac_elem * z_elem)
    
        lgamma = np.zeros_like(flux)
        
        for i in range(len(flux)):

            # get the maximum rigidity from the power requirement
            Rmax = max_energy(flux[i], v_over_c=0.5)

            # normalisation of cosmic ray distribution depends on BETA 
            if BETA == 2:
                dynamic = 1.0/(np.log(Rmax/R0))
            else:
                dynamic = (2 - BETA) / ((Rmax/R0)**(2-BETA)-1.0)
                
            # put 10% of jet energy into CRs, store in eV units 
            Q0 = 0.1 * flux[i] / EV2ERGS

            # get the time step - IMPROVE 
            delta_t = tbin * 1000.0 * YR
            
            if plot_all:
                plt.clf()
                fig = plt.figure()
                ax1 = fig.add_subplot(211)
                ax1.plot(lc.time[:i+1] / 1000.0, flux[:i+1])
                ax1.set_yscale("log")
                ax1.set_xlabel("t (Myr)")
                ax1.set_ylabel("$Q_j$ (erg/s)")
            
            lcr = 0.0

            for j, frac in enumerate(frac_elem):

                rigidities = energies / z_elem[j]

                # escape time is 1e7 YR for 10EV rigidity 
                escape_time = (1e19 / rigidities) * 1e7 * YR 

                # need to get things in the right units
                nu_i = frac / z_elem[j] / R0**2 * dynamic / Zbar

                # add_on is the injection term 
                add_on = nu_i * Q0 * ((rigidities / R0)**-BETA)

                # beyond the cutoff we set injected term to 0
                add_on[(rigidities>=Rmax)] = 0.0
                add_on[(rigidities<=R0)] = 0.0

                # number of escaping CRs 
                escaping = ncr[j] / escape_time

                # loss rate 
                loss =  ncr[j] / (tau_loss[elems[j]].total_interpol *  1e6 * YR)

                # this is dn/dt for each energy 
                change = (add_on - escaping - loss)*delta_t

                # change the ncr distribution
                ncr[j] += change
                ncr[j][(ncr[j]<0)] = 0

                #if j == 0:
                ncr_time[j,i,:] = ncr[j]
                escaping_time[j,i,:] = escaping

                # cutoff for UHECRs (60 EeV)
                select = (energies > 6e19)

                # store the UHECR luminosity 
                lcr += np.fabs(np.trapz(energies[select] * EV2ERGS, energies[select] * escaping[select]))
                
                if j == 0:
                    #lcrtot = np.trapz(energies * EV2ERGS, energies * escaping)
                    all_lobe = np.trapz(energies * EV2ERGS, energies * ncr[j])
                    #print (lcr, lcrtot, all_lobe)

                if plot_all:
                    ax2 = fig.add_subplot(223)
                    energy = rigidities * z_elem[j]
                    ax2.plot(energy, energy*energy*ncr[j])
                    plt.loglog()
                    ax2.set_xlim(1e17,1e21)
                    ax2.set_ylim(1e70,1e75)

                    ax3 = fig.add_subplot(224)
                    plt.plot(energy, energy*energy*escaping*delta_t)

                    # plt.plot(energies, 1e15*((energies / 1e9)**-2), c="k", ls="--")
                    ax2.set_xlim(1e17,1e21)
                    ax2.set_ylim(1e67,1e72)
                    #plt.plot(energies, 1e43*((energies / 1e9)**-3), c="k", ls=":")
                    plt.loglog()
            
            if plot_all:
                plt.savefig("spectra/spectra{:03d}.png".format(i))
                plt.close("all")
        
        # get approximate gamma ray luminosity around 10 GeV
        select = (energies > 1e10) * (energies < 2e10)
        my_lgamma = np.fabs(np.trapz(energies * EV2ERGS, energies*ncr[0]))
        
        # distance to Cen A
        distance = 3.7 * PARSEC * 1e6
        lgammas[i_flux,i_beta] = my_lgamma * 1e-4 * 4.5e-26 * 0.5 * C / 4.0 / PI / distance / distance
        lcrs[i_flux,i_beta] = lcr
        
        print ("BETA {:.1f} median lum {:8.4e} mean lum {:8.4e} UHECR LUM: {:8.4e} {:8.4e}".format(BETA, flux_scale, np.mean(flux), lcr, lgammas[i_flux,i_beta]))


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:124: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:110: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:117: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and

In [ ]:
# Now let's make a movie with ffmpeg (hacky?)
import os
os.system("ffmpeg -y -i spectra/spectra%3d.png -r 6 spectra_over_time.mp4")
import time
time.sleep(3)

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open('spectra_over_time.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

from IPython.display import Video

Video("spectra_over_time.mp4")